**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [2]:
import datetime
from pathlib import Path
import sys
import math

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data, get_male_names

# The Cat Lady

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/5/

“Yes, I did have that tapestry for a little bit. I even cleaned a blotchy section that turned out to be a friendly koala.

“But it was still really dirty, so when I was going through a Marie Kondo phase, I decided it wasn’t sparking joy anymore.

“I listed it on Freecycle, and a woman in **Staten Island** came to pick it up. She was wearing a ‘Noah’s Market’ **sweatshirt, and it was just covered in cat hair**. When I suggested that a clowder of cats might ruin such a fine tapestry, she looked at me funny. She said “I only have **ten or eleven cats, and anyway they are getting quite old now**, so I doubt they’d care about some old rug.”

“It took her 20 minutes to stuff the tapestry into some plastic bags she brought because it was raining. I spent the evening cleaning my apartment.”

What’s the phone number of the woman from Freecycle?

In [4]:
data = get_data(data_path=Path('./../data/5784/'))

df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

df_senior_cat_food = df_products.select(pl.col(["sku", "desc"])).filter(
    (pl.col("sku").str.starts_with("PET"), 
     pl.col("desc").str.contains("Senior"),
    pl.col("desc").str.contains("Cat"),
    )
)

df_female_staten_island = df_customers.filter(
    pl.col("citystatezip").str.starts_with("Staten Island")
)

df_female_staten_island_cat_food_orders = (
    df_senior_cat_food.join(df_orders_items, on="sku")
    .join(df_orders, on="orderid")
    .join(df_female_staten_island, on="customerid")
    .select(
        [
            "name",
            "orderid",
            "qty",
            "customerid",
            "ordered",
            "birthdate",
            "phone",
            "sku",
            "desc",
        ]
    )
)

pl.Config.set_tbl_rows(n=1000)
sol_customer_id = (
    df_female_staten_island_cat_food_orders
    .select(["name", "phone", "customerid"])
    .join(df_orders, on="customerid")
    .join(df_orders_items, on="orderid")
    .join(df_products, on="sku")
    .select(["customerid", "qty"])
    .group_by(["customerid"])
    .agg(pl.col("qty").sum())
    .sort(by="qty", descending=True)
    )

# print(df_customers.select(["customerid", "name", "phone"]).filter(pl.col("customerid") == sol_customer_id).get_column("phone").item())
sol_customer_id


customerid,qty
i64,i64
3068,3549
2345,174
5288,112
4795,96
2996,84
3408,66
7762,60
2094,56
6284,51
